In [2]:
import numpy as np
# Model/layer imports
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GRU, Dense, Dropout, SpatialDropout1D
from tensorflow.keras.losses import sparse_categorical_crossentropy

In [3]:
#If using GPU, reset memory allocation, and assign GPU with memory growth.
tf.compat.v1.reset_default_graph()
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

In [ ]:
#Open War and Peace
path_to_file = "tolstoybot.txt"
text = open(path_to_file, 'r', encoding='utf-8').read()

In [ ]:
#Obtain unique vocab set and size 
vocab = sorted(set(text))
vocab_size = len(vocab)

In [ ]:
#Create an index mapping for characters in vocab (bi-directional)
char_to_ind = {char:ind for ind, char in enumerate(vocab)}
ind_to_char = np.array(vocab)

In [ ]:
#Encode whole of text file using vocab index map
encoded_text = np.array([char_to_ind[c] for c in text])

In [ ]:
# #Query line(s) length(s)

# line = 'Well, Prince, so Genoa and Lucca are now just family estates of the Buonapartes.'
# print(len(line))

# lines='''
# But I warn you, if you don’t tell me that this means war,
# if you still try to defend the infamies and horrors perpetrated by that
# Antichrist—I really believe he is Antichrist—I will have nothing
# more to do with you and you are no longer my friend, no longer my
# ‘faithful slave,’ as you call yourself!
#   '''
# print(len(lines))

In [ ]:
#Set length of sequence
seq_len = 128

In [ ]:
#Determine total number of sequences in text
total_num_seq = len(text) // (seq_len+1)

In [ ]:
#Create a Character Dataset from encoded text 
char_dataset = tf.data.Dataset.from_tensor_slices(encoded_text)

In [ ]:
#Combine into sequences
sequences = char_dataset.batch(seq_len+1, drop_remainder=True)

In [ ]:
# Create method for splitting sequences into input/target 
def create_seq_targets(seq):
    input_txt = seq[:-1]
    target_txt = seq[1:]
    return input_txt, target_txt

#Use method to create complete training dataset
dataset = sequences.map(create_seq_targets)

In [ ]:
#Set batch size
batch_size = 32
#Set buffer size
buffer_size = 10000

In [ ]:
#Shuffle the dataset
dataset = dataset.shuffle(buffer_size).batch(batch_size,drop_remainder=True)

In [ ]:
# #view shape of dataset
#dataset

In [ ]:
#Set embedding dimensions
embed_dim = 256

In [ ]:
#Choose amount of neurons in GRU layers
rnn_neurons4 = 1024
rnn_neurons3 = 512
rnn_neurons2 = 256
rnn_neurons = 105

In [ ]:
#Customised Sparse_cat_cross loss function 
def sparse_cat_loss(y_true,y_pred):
    return sparse_categorical_crossentropy(y_true, y_pred, from_logits=True)

In [ ]:
#Function for creating model
def create_model(vocab_size, embed_dim,rnn_neurons,rnn_neurons2,rnn_neurons3,rnn_neurons4,batch_size):
    model = Sequential()
    model.add(Embedding(vocab_size, embed_dim, batch_input_shape=[batch_size,None]))
    model.add(SpatialDropout1D(0.2))
    
    model.add(GRU(rnn_neurons, return_sequences=True, 
                 stateful=True, recurrent_initializer='glorot_uniform', batch_input_shape=[batch_size,None, embed_dim]))

    model.add(Dropout(0.2))
    
    model.add(GRU(rnn_neurons2, return_sequences=True, 
                 stateful=True, recurrent_initializer='glorot_uniform'))

    model.add(Dropout(0.2))
    
    model.add(GRU(rnn_neurons3, return_sequences=True, 
                 stateful=True, recurrent_initializer='glorot_uniform'))

    model.add(Dropout(0.2))
    
    model.add(GRU(rnn_neurons4, return_sequences=True, 
                 stateful=True, recurrent_initializer='glorot_uniform'))
    
    model.add(Dropout(0.2))
    
    model.add(Dense(vocab_size))
    model.compile('adam', loss=sparse_cat_loss, metrics=['accuracy'])
    return model

In [ ]:
# Create training model
model = create_model(vocab_size=vocab_size,
                     embed_dim=embed_dim,
                     rnn_neurons=rnn_neurons,
                     rnn_neurons2=rnn_neurons2,
                     rnn_neurons3=rnn_neurons3,
                     rnn_neurons4=rnn_neurons4,
                     batch_size=batch_size)

In [ ]:
# Display model summary
model.summary()

In [ ]:
#Set epochs 
epochs = 100

In [ ]:
#Create early stopping function
early_stop = EarlyStopping(monitor='loss', patience=3)

In [ ]:
#Fit the model
model.fit(dataset,epochs=epochs, callbacks=[early_stop])

In [ ]:
#Save the model
model.save('tolstoybot_train.h5')

In [ ]:
#Create a new version of the model (test_model) with a single input batch size
test_model = create_model(vocab_size,embed_dim,rnn_neurons, rnn_neurons2, rnn_neurons3, rnn_neurons4, batch_size=1)
test_model.load_weights('tolstoybot_train.h5')
test_model.build(tf.TensorShape([1,None]))
test_model.save('tolstoybot_test.h5')

In [ ]:
###RUN THE BELOW CODE IF YOU ARE OPENING THIS AFTER TRAINING###

In [4]:
# #If loading seperately
# import tensorflow as tf
# import numpy as np
# from tensorflow.keras.models import load_model

# # #if using GPU, set the correct GPU and assign memory growth
# # tf.compat.v1.reset_default_graph()
# # physical_devices = tf.config.list_physical_devices('GPU')
# # tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

# #Open tolstoybot.txt
# path_to_file = "tolstoybot.txt"
# text = open(path_to_file, 'r', encoding='utf-8').read()

# #Obtain unique vocab set and size 
# vocab = sorted(set(text))
# vocab_size = len(vocab)

# #Create an index mapping for characters in vocab (bi-directional)
# char_to_ind = {char:ind for ind, char in enumerate(vocab)}
# ind_to_char = np.array(vocab)

# #Load the model to memory
# test_model = load_model('tolstoybot_test.h5', compile=False)

In [5]:
###RUN THE ABOVE CODE IF YOU ARE OPENING THIS AFTER TRAINING###

In [6]:
# Display test model summary
test_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            26880     
_________________________________________________________________
spatial_dropout1d_1 (Spatial (1, None, 256)            0         
_________________________________________________________________
gru_4 (GRU)                  (1, None, 105)            114345    
_________________________________________________________________
dropout_4 (Dropout)          (1, None, 105)            0         
_________________________________________________________________
gru_5 (GRU)                  (1, None, 256)            278784    
_________________________________________________________________
dropout_5 (Dropout)          (1, None, 256)            0         
_________________________________________________________________
gru_6 (GRU)                  (1, None, 512)           

In [7]:
#Test function for generating consecutive text
def generate_text(model,start_seed,num_generate=500,temperature=1):
    
    #Map each character in start_seed to it's relative index
    input_eval = [char_to_ind[s] for s in start_seed]
    
    #Expand the dimensions
    input_eval = tf.expand_dims(input_eval, 0)
    
    #Create empty list for generated text
    text_generated = []
    
    #Reset the states of tyhe model        
    model.reset_states()
    
    #for each iteration of num_generate
    for i in range(num_generate):
        
        #Obtain probability matrix for current iteration 
        predictions = model(input_eval)
        
        #Reduce dimensions
        predictions = tf.squeeze(predictions,0)
        
        #Multiply probabily matrix by temperature
        predictions = predictions/temperature
        
        #Select a random outcome, based on the unnormalised log-probabilities produced by the model
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
        
        #Expand dimensions of prediction and assign as next input evaluation
        input_eval = tf.expand_dims([predicted_id], 0)
        
        #Convert prediction to char and append to generated list of text
        text_generated.append(ind_to_char[predicted_id])
        
    #return the initial input, concatenated with the generated text. 
    return(start_seed+"".join(text_generated))

In [8]:
# Note: Temperature affects the probability of characters chosen, should range between 0.1 and 2
# Note: A smaller temperature will result in more predictable text!

In [9]:
# Test the model with sample text 
print(generate_text(test_model,"This is a test", num_generate=500, temperature=0.5))

This is a testain pretending to ask him to say that he had reached the
Emperor who was the soldier of the people with the movement of the conception of
the country. The last days were being led him and go to the cause of the
army, and the count’s hussar and such full of strength and considerations and
the people are essential to the Emperor and have been in the same man days in the
conception of the character in the army because they have been
received and she went out of the way had been in the commander in 
